In [8]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go



Weekly & Monthly Revenue Trends (Whole market)


In [9]:
# Load data from the Weekly sheet
weekly_data = pd.read_excel('Customer_Success_Engineer_-_Project_Data.xlsx', sheet_name='Weekly', header=1)

# Load data from the Monthly sheet
monthly_data = pd.read_excel('Customer_Success_Engineer_-_Project_Data.xlsx', sheet_name='Monthly', header=1)

# Processing for weekly_data
weekly_data['Week'] = weekly_data['Week'].str.split(' - ').str[0].str.strip()
weekly_data['Week'] = pd.to_datetime(weekly_data['Week'], errors='coerce')
weekly_data.sort_values('Week', inplace=True)
weekly_revenue = weekly_data.groupby('Week')['Revenue'].sum().reset_index()

# Processing for monthly_data
# Combining 'Year' and 'Month' to form a datetime object
monthly_data['Month'] = pd.to_datetime(monthly_data['Year'].astype(str) + '-' + monthly_data['Month'], format='%Y-%B', errors='coerce')
monthly_data.sort_values('Month', inplace=True)
monthly_revenue = monthly_data.groupby('Month')['Revenue'].sum().reset_index()

# Plotting weekly revenue trends
fig_weekly = px.line(weekly_revenue, x='Week', y='Revenue',
              title='Time Series Analysis of Weekly Revenue Trends',
              labels={'Revenue': 'Total Revenue', 'Week': 'Date'})

fig_weekly.update_layout(xaxis_title='Date', yaxis_title='Revenue ($)')
fig_weekly.show()

# Plotting monthly revenue trends
fig_monthly = px.line(monthly_revenue, x='Month', y='Revenue',
              title='Time Series Analysis of Monthly Revenue Trends',
              labels={'Revenue': 'Total Revenue', 'Month': 'Date'})

fig_monthly.update_layout(xaxis_title='Date', yaxis_title='Revenue ($)')
fig_monthly.show()


Market Share Distribution by brand - groups up all brands with lower than 1% market share

In [10]:
# Load brand data
brand_data = pd.read_excel('Customer_Success_Engineer_-_Project_Data.xlsx', sheet_name='Segment Brands', header=1)

# Summarize total revenue by brand
brand_revenue = brand_data.groupby('Brand')['Revenue'].sum().reset_index()

# Calculate market share
brand_revenue['Market Share'] = (brand_revenue['Revenue'] / brand_revenue['Revenue'].sum()) * 100

# Define a threshold for minimum market share to be shown individually
threshold = 1  # for example, 1%

# Create a new row for 'Other' by summing up brands below the threshold
other_revenue = brand_revenue[brand_revenue['Market Share'] < threshold]['Revenue'].sum()
other_market_share = brand_revenue[brand_revenue['Market Share'] < threshold]['Market Share'].sum()

# Remove brands below the threshold
brand_revenue = brand_revenue[brand_revenue['Market Share'] >= threshold]

# Append the 'Other' category to the DataFrame
other_row = pd.DataFrame(data={'Brand': ['Other'], 'Revenue': [other_revenue], 'Market Share': [other_market_share]})
brand_revenue = pd.concat([brand_revenue, other_row], ignore_index=True)

# Sort by market share to ensure 'Other' is not dominating the chart if it's a large share
brand_revenue.sort_values('Market Share', inplace=True)

# Create a pie chart
fig = px.pie(brand_revenue, values='Market Share', names='Brand', title='Market Share Distribution by Brand')

# Show the pie chart
fig.show()

Brand Performance Comparison

Grouping the data by 'Brand' to calculate the sum of revenues for each.
Sorting the results to ensure that the bar chart shows brands in order of their revenue performance.
Plotting the data using Plotly's bar chart function.

In [11]:
# Load brand data from the Excel sheet
brand_data = pd.read_excel('Customer_Success_Engineer_-_Project_Data.xlsx', sheet_name='Segment Brands', header=1)

# Summarize total revenue by brand (assuming 'Brand' is the correct column name)
brand_performance = brand_data.groupby('Brand')['Revenue'].sum().reset_index()

# Sort the DataFrame based on revenue in descending order for better visualization
brand_performance = brand_performance.sort_values('Revenue', ascending=False)

# Create the bar chart
fig = px.bar(brand_performance, x='Brand', y='Revenue', title='Brand Performance Comparison')

# Improve the layout
fig.update_layout(
    xaxis_title='Brand',
    yaxis_title='Total Revenue ($)',
    xaxis={'categoryorder':'total descending'},  # This ensures the bars are sorted by revenue
    yaxis=dict(type='log')  # Using a log scale for the y-axis can be helpful if there are large differences in revenue
)

# Show the figure
fig.show()

Average Price Trending Over Time

In [12]:
# Load data from the 'Weekly' sheet
weekly_data = pd.read_excel('Customer_Success_Engineer_-_Project_Data.xlsx', sheet_name='Weekly', header=1)

# Assuming 'Week' is the column with date information and 'Price' is the column with the product prices
# Convert 'Week' to datetime (adjust the string split if the date format is different)
weekly_data['Week'] = pd.to_datetime(weekly_data['Week'].str.split(' - ').str[0])

# Group by 'Week' and calculate the average price
weekly_avg_price = weekly_data.groupby('Week')['Price'].mean().reset_index()

# Create the line chart
fig = px.line(weekly_avg_price, x='Week', y='Price', title='Average Price Trending Over Time')

# Improve the layout
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Average Price ($)',
    xaxis=dict(
        tickmode='auto',  # you can change to 'linear' if you want even spacing regardless of the actual time gaps
        nticks=20  # Adjust the number of ticks to get a clear view of the data points
    )
)

# Show the figure
fig.show()

Unit Sales Over Time for Top Brands

1. Load the data from the 'Weekly' or 'Monthly' dataset, depending on the desired time granularity.
2. Select the top brands based on total unit sales.
3. Group the data by the time period and brand, then sum up the unit sales for each period for each of the  top brands.
4. Plot the trends using a multi-line chart, with each line representing one of the top brands.

In [13]:
# Load the 'Weekly' data, assuming 'Weekly' data has the unit sales and 'Brand' information
sales_data = pd.read_excel('Customer_Success_Engineer_-_Project_Data.xlsx', sheet_name='Weekly', header=1)

# Convert 'Week' to datetime (the following is a basic method; you may need to adjust it based on your actual data format)
sales_data['Week'] = pd.to_datetime(sales_data['Week'].str.split(' - ').str[0])

# Identify the top brands by total unit sales
top_brands = sales_data.groupby('Brand')['Units'].sum().nlargest(5).index

# Filter the data to include only the top brands
top_brands_data = sales_data[sales_data['Brand'].isin(top_brands)]

# Group by 'Week' and 'Brand' and sum up unit sales
unit_sales_over_time = top_brands_data.groupby(['Week', 'Brand'])['Units'].sum().reset_index()

# Pivot the data to have 'Week' as index and the unit sales of each top brand as a column
unit_sales_pivot = unit_sales_over_time.pivot(index='Week', columns='Brand', values='Units')

# Reset the index to turn 'Week' back into a column
unit_sales_pivot.reset_index(inplace=True)

# Melt the DataFrame to have a single 'Units' column with corresponding 'Brand' and 'Week'
unit_sales_melted = unit_sales_pivot.melt(id_vars='Week', var_name='Brand', value_name='Units')

# Plotting the multi-line chart
fig = px.line(unit_sales_melted, x='Week', y='Units', color='Brand', title='Unit Sales Over Time for Top Brands')

# Improve the layout
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Unit Sales',
    legend_title='Brand'
)

# Show the figure
fig.show()